In [1]:
import xlrd
import xlwt
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
import neurolab as nl

import pandas as pd
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import time

data = pd.read_excel('data1.xlsx')

del data['FROM DATE']
del data['TO DATE']
del data['VWS']
del data['TEMP']

names = data.columns



data.drop_duplicates(inplace=True)  # removing duplicates



limit = {}


def IQR_outlier(dt, name):
    q1 = dt.quantile(.25)
    q3 = dt.quantile(.75)
    iqr = q3 - q1
    l_limit = q1 - 1.5 * iqr
    r_limit = q3 + 1.5 * iqr
    l_limit = round(l_limit, 2)
    r_limit = round(r_limit, 2)
    limit[name] = [l_limit, r_limit]


def removal_outlier(st, name):
    st = st[st < limit[name][1]]
    st = st[st > limit[name][0]]
    return st


# for i in range(len(names)):
    # IQR_outlier(data[names[i]], names[i])

# for i in range(len(names)):
    # data[names[i]] = removal_outlier(data[names[i]], names[i])

data = data.dropna()


In [2]:
data


,AT,BP,PM10,PM2.5,RH,SR,WD,WS,BENZENE,TOLUENE,NH3,NO2,NO,NOX,SO2,OZONE,CO
0,23.77,988.86,492.86,200.39,69.06,59.00,137.95,0.62,11.51,88.66,58.04,62.97,93.09,110.74,25.02,43.10,3.31
1,25.39,987.78,444.95,187.33,55.34,73.86,114.81,0.74,6.84,68.06,51.26,58.65,36.16,60.70,33.30,40.96,2.40
2,24.57,986.17,354.62,158.21,59.26,40.93,90.52,1.20,4.40,29.94,52.46,38.06,7.78,26.58,26.00,61.63,1.53
3,23.10,987.92,213.85,103.95,54.98,62.31,202.50,1.17,2.54,17.17,34.73,22.71,3.22,14.71,26.02,85.91,1.03
4,19.08,988.91,477.14,320.24,75.92,73.74,277.36,0.91,4.93,27.30,45.86,32.27,17.83,31.74,23.62,31.63,1.89
5,19.87,989.12,348.94,181.70,53.99,73.28,280.83,1.00,3.35,19.67,30.09,26.43,14.07,25.53,23.20,40.53,1.28
6,18.24,989.34,338.10,236.92,48.09,59.67,271.31,0.71,4.29,26.70,26.61,28.73,32.88,42.10,31.58,42.64,1.70
7,18.06,989.58,781.34,453.22,65.05,57.62,201.25,0.49,8.64,58.90,39.84,45.03,94.55,101.14,40.28,24.29,3.27
8,19.40,988.09,563.16,326.55,74.53,75.03,257.75,0.48,11.53,66.10,53.93,45.47,103.07,107.48,26.87,30.16,3.70
9,19.80,988.82,546.48,279.36,77.22,64.57,178.95,0.49,9.43,77.87,59.94,53.21,84.94,97.59,27.95,26.71,3.38
